# 메모리

`langgraph`에서 메모리는 상태 그래프 자체에 의해 관리됩니다. 상태의 각 단계는 체크포인트로 저장할 수 있습니다. 그런 다음 언제든지 특정 체크포인트에서 실행을 재개할 수 있습니다.

## 설정

먼저 필요한 패키지를 설치합니다.

In [ ]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai

다음으로, 환경 변수를 설정합니다.

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()

## 체크포인터

메모리를 활성화하려면 체크포인터를 전달해야 합니다. 체크포인터는 그래프의 상태를 저장하는 역할을 합니다. 현재 `langgraph`에는 몇 가지 다른 유형의 체크포인터가 있습니다:

1. `MemorySaver`: 상태를 인메모리 딕셔너리에 저장합니다.
2. `AsyncSqliteSaver`: 상태를 SQLite 데이터베이스에 저장합니다.

이 예제에서는 `MemorySaver`를 사용하겠습니다.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

## 그래프 정의

이제 체크포인터를 사용하여 그래프를 정의할 수 있습니다. 이전 노트북과 동일한 에이전트를 사용하지만, 이번에는 `checkpointer` 인자를 전달하여 메모리를 활성화합니다.

In [ ]:
from typing import Annotated

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

tool = TavilySearchResults(max_results=2)
tools = [tool]
llm = ChatOpenAI(model="gpt-4o")
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    lambda x: "tools" if x["messages"][-1].tool_calls else "__end__",
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")

# checkpointer를 전달하여 메모리를 활성화합니다.
graph = graph_builder.compile(checkpointer=memory)

## 그래프 사용

이제 그래프를 사용할 수 있습니다. `configurable` 인자를 전달하여 각 대화에 대한 스레드를 지정해야 합니다. 이는 `langgraph`가 대화 기록을 올바르게 저장하고 로드하는 데 사용됩니다.

In [ ]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

user_input = "안녕, 내 이름은 윌이야. 랭체인에 대해 뭔가 가르쳐줄 수 있니?"

# stream() 또는 invoke()를 호출할 때마다
# 중간 단계가 체크포인터에 저장됩니다.
events = graph.stream(
    {"messages": [HumanMessage(content=user_input)]},
    config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

이제 후속 질문을 할 수 있습니다. `langgraph`는 이전 메시지를 기억하고 이를 사용하여 응답합니다.

In [ ]:
user_input = "내 이름이 뭐라고 했지?"
events = graph.stream(
    {"messages": [HumanMessage(content=user_input)]}, config, stream_mode="values"
)
for event in events:
    event["messages"][-1].pretty_print()

## 상태 보기

체크포인터에서 직접 상태를 볼 수도 있습니다.

In [ ]:
graph.get_state(config)

## 상태 수정

체크포인터를 사용하여 상태를 수정할 수도 있습니다. 예를 들어, 대화 기록을 수정하여 에이전트가 다른 방식으로 응답하도록 할 수 있습니다.

In [ ]:
graph.update_state(config, {"messages": [HumanMessage(content="내 이름은 윌이 아니라 밥이야")]})

In [ ]:
graph.get_state(config)

In [ ]:
user_input = "내 이름이 뭐라고 했지?"
events = graph.stream(
    {"messages": [HumanMessage(content=user_input)]}, config, stream_mode="values"
)
for event in events:
    event["messages"][-1].pretty_print()

## 다음

다음으로, 상태를 수정하는 또 다른 방법인 중단점에 대해 알아보겠습니다.